Imports and Global variables

In [30]:
import numpy as np
import pandas as pd
import anndata as ad

import os

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn import preprocessing
from sklearn.decomposition import PCA

neural_color_dict = {'strong neural':'darkorange','neural':'gold','weak neural':'khaki','neutral':'darkgray','weak stem':'lightskyblue','stem':'deepskyblue','strong stem':'royalblue', 'undetermined':'mediumspringgreen'}
seqtypes = ['arm.smallncRNAs']

# 1. Build AnnData Objects

The following code will build the AnnData objects from the tRAX run as described in the [README](README.md)

In [ ]:
for seqtype in seqtypes:
    ! ../tRNAgraph/trnagraph.py build -i trax/{seqtype} -m config/metadata.tsv -o {seqtype}.h5ad

# 2. Manual addition of metadata

Modification data is manually added to the AnnData objects for later analysis via clustering.

In [3]:
for seqtype in seqtypes:
    # Load the adata
    adata = ad.read_h5ad(f'{seqtype}.h5ad')
    # Load the mods table
    mods_df = pd.read_csv('supplemental/ModTable_Human_labeled.txt', sep='\t')
    mods_df.rename(columns={'Unnamed: 0': 'trna'}, inplace=True)
    # Add the mods to the adata.obs individually
    for i in mods_df.columns[1:-1]:
        mods_dict = dict(zip(mods_df['trna'], mods_df[i]))
        adata.obs['mods_'+i] = adata.obs['trna'].map(mods_dict)

    # Add fancy mod names for plotting as uns dict
    mod_fancy_names_dict = {'m2G_6':r'm$^2$G 6', 'm2G_7':r'm$^2$G 7', 'm1A_9':r'm$^1$A 9', 'm1G_9':r'm$^1$G 9', 'm2G_10':r'm$^2$G 10', 'm1A_14':r'm$^1$A 14', 'acp3U_20':r'acp$^3$U 20', \
                        'm3C_20':r'm$^3$C 20', 'acp3U_20a':r'acp$^3$U 20a', 'm3C_20a':r'm$^3$C 20a', 'm22G_26':r'm$^2$$_2$G 26', 'm2G_26':r'm$^2$G 26', 'm22G_27':r'm$^2$$_2$G 27', \
                        'm3C_32':r'm$^3$C 32', 'I_34':r'I 34', 'm1A_37':r'm$^1$A 37', 'm1G_37':r'm$^1$G 37', 'm1I_37':r'm$^1$I 37', 'm6t6A_37':r'm$^6t^6$A 37', 'ms2t6A_37':r'ms$^2t^6$A 37', \
                        'm3C_e2':r'm$^3$C e2', 'm1A_58':r'm$^1$A 58'}
    adata.uns['mod_fancy_names'] = mod_fancy_names_dict
    
    # Save the adata
    adata.write_h5ad(f'{seqtype}.h5ad')

Add log2FC and p-value data to the AnnData objects for AlkBp filtering.

In [ ]:
for seqtype in seqtypes:
    !../tRNAgraph/trnagraph.py tools log2fc -i {seqtype}.h5ad --cutoff 20 60 80 --config config/AlkBp.json --quiet

Load the covariance, hmm, and secondary structure data from gtRNAdb and add it to the AnnData objects.

In [5]:
df = pd.read_csv('../rnadb/hg38-tRNAs-detailed.out', sep='\t', header=2)
# Keep columns 0, 1, 4, 5, 8, 9, 10
df = df.iloc[:, [0, 1, 4, 5, 8, 9, 10]]
df.columns = ['chr', 'trna', 'amino', 'iso', 'coveScore', 'HMM_score', 'SecStruct_score']
# convert trna to gtRNAdb id
name_df = pd.read_csv('../rnadb/hg38-tRNAs_name_map.txt', sep='\t')
name_dict = dict(zip(name_df['tRNAscan-SE_id'], name_df['GtRNAdb_id']))
name_dict = {k:'-'.join(v.split('-')[:-1]) for k, v in name_dict.items()}
df['trna'] = df['chr'] + '.trna' + df['trna'].astype(str)
df['trna'] = df['trna'].map(name_dict)

for seqtype in seqtypes:
    adata = ad.read_h5ad(f'{seqtype}.h5ad')
    # Add HMM score information
    adata.obs['cove_score'] = adata.obs['trna'].map(dict(zip(df['trna'], df['coveScore'])))
    adata.obs['hmm_score'] = adata.obs['trna'].map(dict(zip(df['trna'], df['HMM_score'])))
    adata.obs['secstruct_score'] = adata.obs['trna'].map(dict(zip(df['trna'], df['SecStruct_score'])))
    adata.write_h5ad(f'{seqtype}.h5ad')

Neural and Stem cell markers are added to the AnnData object for later analysis.

In [ ]:
class neuralStem():
    def __init__(self, adata, compare, readtype, colselect):
        self.adata = adata
        self.colselect = colselect
        self.percentiles = [0.1,0.2,0.3,0.5,0.7,0.8,0.9]
        self.pvalcutoff = 0.05
        # Search nested adata.uns for log2FC for compare and readtype
        self.log2fc_dict = self.adata.uns.get('log2FC', {})
        self.df = self.log2fc_dict['AlkBp'][compare][readtype]['20']['df']
        self.pairs = self.log2fc_dict['AlkBp'][compare]['pairs']

    def main(self):
        self.qdf_pos, self.qdf_neg = self.gen_quartile_df()
        self.assign_categories()
        plt = self.plot_log2fc()
        return self.adata, plt

    def gen_quartile_df(self):
        self.df = self.df[[f'log2_{self.colselect}',f'pval_{self.colselect}']]
        self.df = self.df.sort_values(ascending=False, by=f'log2_{self.colselect}')
        # self.df = self.df.to_frame()
        self.df['trna'] = self.df.index.tolist()
        self.df = self.df.reset_index(drop=True)
        # Set column names
        self.df.columns = ['log2FC','pval','trna']
        # Scale the log2FC between -1 and 1
        scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
        self.df['log2FC'] = scaler.fit_transform(self.df['log2FC'].values.reshape(-1,1))
        # Split the df into positive and negative log2FC then calculate the quantile ranges for each before combining
        pos_df = self.df[self.df['log2FC'] >= 0]
        neg_df = self.df[self.df['log2FC'] <= 0]
        pos_df = pos_df.describe(percentiles=self.percentiles[4:])
        neg_df = neg_df.describe(percentiles=self.percentiles[:4])

        return pos_df, neg_df

    def assign_categories(self):
        # Set category based on quantile ranges
        neural_fine_dict, neural_broad_dict = {}, {}
        for i,row in self.df.iterrows():
            if row['log2FC'] > self.qdf_pos.loc[f'{int(self.percentiles[-1]*100)}%','log2FC'] and row['log2FC'] > self.qdf_pos.loc['50%','log2FC'] or row['log2FC'] >= 0.95:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'strong neural'
                    neural_broad_dict[row['trna']] = 'neural'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            elif row['log2FC'] > self.qdf_pos.loc[f'{int(self.percentiles[-2]*100)}%','log2FC'] and row['log2FC'] > self.qdf_pos.loc['50%','log2FC'] or row['log2FC'] >= 0.90:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'neural'
                    neural_broad_dict[row['trna']] = 'neural'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            elif row['log2FC'] > self.qdf_pos.loc[f'{int(self.percentiles[-3]*100)}%','log2FC'] and row['log2FC'] > self.qdf_pos.loc['50%','log2FC'] or row['log2FC'] >= 0.75:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'weak neural'
                    neural_broad_dict[row['trna']] = 'neural'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            elif row['log2FC'] < self.qdf_neg.loc[f'{int(self.percentiles[0]*100)}%','log2FC'] and row['log2FC'] < self.qdf_neg.loc['50%','log2FC'] or row['log2FC'] <= -0.95:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'strong stem'
                    neural_broad_dict[row['trna']] = 'stem'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            elif row['log2FC'] < self.qdf_neg.loc[f'{int(self.percentiles[1]*100)}%','log2FC'] and row['log2FC'] < self.qdf_neg.loc['50%','log2FC'] or row['log2FC'] <= -0.90:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'stem'
                    neural_broad_dict[row['trna']] = 'stem'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            elif row['log2FC'] < self.qdf_neg.loc[f'{int(self.percentiles[2]*100)}%','log2FC'] and row['log2FC'] < self.qdf_neg.loc['50%','log2FC'] or row['log2FC'] <= -0.75:
                if row['pval'] < self.pvalcutoff:
                    neural_fine_dict[row['trna']] = 'weak stem'
                    neural_broad_dict[row['trna']] = 'stem'
                else:
                    neural_fine_dict[row['trna']] = 'neutral'
                    neural_broad_dict[row['trna']] = 'neutral'
                continue
            else:
                neural_fine_dict[row['trna']] = 'neutral'
                neural_broad_dict[row['trna']] = 'neutral'
        self.df['neural_fine'] = self.df['trna'].map(neural_fine_dict)
        self.df['neural_broad'] = self.df['trna'].map(neural_broad_dict)
        self.adata.obs['neural_fine'] = self.adata.obs['trna'].map(neural_fine_dict)
        self.adata.obs['neural_broad'] = self.adata.obs['trna'].map(neural_broad_dict)
        # Replace NaN with neutral
        self.adata.obs['neural_fine'] = self.adata.obs['neural_fine'].fillna('neutral') #.fillna('undetermined')
        self.adata.obs['neural_broad'] = self.adata.obs['neural_broad'].fillna('neutral') #.fillna('undetermined')

        log2fc_dict = dict(zip(self.df['trna'], self.df['log2FC']))
        self.adata.obs['neural_stem_score'] = self.adata.obs['trna'].map(log2fc_dict)
        # Fill NaN with 0
        self.adata.obs['neural_stem_score'] = self.adata.obs['neural_stem_score'].fillna(0)

    def plot_log2fc(self):
        # Plot the slope
        plt.figure(figsize=(6,18))
        sns.barplot(x='log2FC', y='trna', data=self.df, hue='neural_fine', palette=neural_color_dict, dodge=False)
        plt.xlabel('Slope')
        plt.ylabel('tRNA')
        plt.title('Slope of tRNA decay')
        # Move the legend outside the plot
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)
        plt.tight_layout()
        return plt

seqtype = 'arm.smallncRNAs'
adata = ad.read_h5ad(f'{seqtype}.h5ad')
adata, plt = neuralStem(adata, 'group', 'nreads_total_unique_norm', f'AlkBp_arm_d0-AlkBp_arm_d70').main()
os.makedirs(f'figures/{seqtype}/neuralstem', exist_ok=True)
plt.savefig(f'figures/{seqtype}/neuralstem/slope.pdf')
# plt.close()
adata.write_h5ad(f'{seqtype}.h5ad')

# 3. Clustering and HDBSCAN annotation

The following code will run the cluster part of the pipeline on the ARMseq data with the following parameters:

In [ ]:
randomstate = 27
readcutoff = 15

coveragetype = ' '.join(['uniquecoverage', 'readstarts', 'readends', 'mismatchedbases', 'deletions'])

sample_neighbors_cluster = 150
sample_neighbors_plot = 75
sample_hdbscan_min_samples = 6
sample_hdbscan_min_cluster_size = 30
sample_n_components = 10

group_neighbors_cluster = 35
group_neighbors_plot = 18
group_hdbscan_min_samples = 4
group_hdbscan_min_cluster_size = 12
group_n_components = 10

seqtype = 'arm.smallncRNAs'
! ../tRNAgraph/trnagraph.py cluster -i {seqtype}.h5ad -o {seqtype}.h5ad \
    -r {randomstate} -t {readcutoff} -v {coveragetype} \
    -c1 {sample_n_components} -c2 {group_n_components} \
    -l1 {sample_neighbors_cluster} -l2 {group_neighbors_cluster} \
    -n1 {sample_neighbors_plot} -n2 {group_neighbors_plot} \
    -d1 {sample_hdbscan_min_samples} -d2 {group_hdbscan_min_samples} \
    -b1 {sample_hdbscan_min_cluster_size} -b2 {group_hdbscan_min_cluster_size} \
    --overwrite --quiet

Determine the percentage of stem/neural tRNAs in each cluster.

In [ ]:
adata = ad.read_h5ad('arm.smallncRNAs.h5ad')
adata = adata[~adata.obs['amino'].isin(['Und','Sup'])]
# Add the neural_stem_score to the adata
stem_neural_percent = {}
for i in adata.obs['group_cluster'].dropna().unique():
    tts = adata.obs[adata.obs['group_cluster'] == i][['nreads_total_unique_norm','neural_broad']].groupby('neural_broad', observed=True).describe()
    tts['group_distribution'] = tts['nreads_total_unique_norm']['count'] / tts['nreads_total_unique_norm']['count'].sum()
    tts['expression_distribution'] = tts['nreads_total_unique_norm']['mean'] / tts['nreads_total_unique_norm']['mean'].sum()
    # Check if steam, neural and neutral rows exist and if not add them with 0
    for j in ['stem','neural','neutral']:
        if j not in tts.index:
            tts.loc[j] = [0,0,0,0,0,0,0,0,0,0]
    stem_neural_percent[i] = (tts['group_distribution']['neural'], tts['group_distribution']['stem'], tts['group_distribution']['neutral'], \
                              tts['expression_distribution']['neural'], tts['expression_distribution']['stem'], tts['expression_distribution']['neutral'])
# Map this back to the adata
adata.obs['group_cluster_neural_dist'] = adata.obs['group_cluster'].map({k:v[0] for k,v in stem_neural_percent.items()})
adata.obs['group_cluster_stem_dist'] = adata.obs['group_cluster'].map({k:v[1] for k,v in stem_neural_percent.items()})
adata.obs['group_cluster_neutral_dist'] = adata.obs['group_cluster'].map({k:v[2] for k,v in stem_neural_percent.items()})
adata.obs['group_cluster_neural_expression'] = adata.obs['group_cluster'].map({k:v[3] for k,v in stem_neural_percent.items()})
adata.obs['group_cluster_stem_expression'] = adata.obs['group_cluster'].map({k:v[4] for k,v in stem_neural_percent.items()})
adata.obs['group_cluster_neutral_expression'] = adata.obs['group_cluster'].map({k:v[5] for k,v in stem_neural_percent.items()})
adata.write_h5ad('arm.smallncRNAs.h5ad')

Assign catgorical labels to each cluster based on the percentage of stem/neural tRNAs in each cluster as well as create a name dictionary for each cluster.

In [ ]:
# adata = ad.read_h5ad('arm.smallncRNAs.h5ad')
df = adata.obs[['group_cluster','neural_stem_score','group_cluster_neural_dist','group_cluster_stem_dist','group_cluster_neutral_dist'\
                ,'group_cluster_neural_expression','group_cluster_stem_expression','group_cluster_neutral_expression']]
# Drop nan and duplicates
df = df.dropna()
df.drop_duplicates(inplace=True)
# grouby group_cluster and get the mean of neural_stem_score
df = df.groupby('group_cluster').mean()
# Scale the neural_stem_score between -1 and 1
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
df['neural_stem_score'] = scaler.fit_transform(df['neural_stem_score'].values.reshape(-1,1))
# df['group_cluster_neural_percent'] = scaler.fit_transform(df['group_cluster_neural_percent'].values.reshape(-1,1))
# Sort by neural_stem_score
# df.sort_values(by='neural_stem_score', ascending=False, inplace=True)
df['group_cluster_stem_expression'] = df['group_cluster_stem_expression'] * -1
df.sort_values(by=['group_cluster_neural_expression','group_cluster_stem_expression'], ascending=False, inplace=True)
# Create an alt_neural_stem_score by combining the neural and stem expression
df['neural_stem_score_alt'] = df['group_cluster_neural_expression'] + df['group_cluster_stem_expression']
# Scale the neural_stem_score between -1 and 1
df['neural_stem_score_alt'] = scaler.fit_transform(df['neural_stem_score_alt'].values.reshape(-1,1))
# Add to the adata
neural_stem_score_alt_dict = dict(zip(df.index, df['neural_stem_score_alt']))
adata.obs['neural_stem_score_alt'] = adata.obs['group_cluster'].map(neural_stem_score_alt_dict)
df['group_cluster_stem_expression'] = df['group_cluster_stem_expression'] * -1
# Determine if neural, stem, neutral based on the relative percentage
cluster_names_dict = {17.:'B1',1.:'B2', 0.:'B3',15.:'B4',19.:'C14',16.:'C13',13.:'C12',14.:'C11',11.:'C10',8.:'C9',-1.:'U',2.:'C8',6.:'C7',10.:'C6',4.:'C5',18.:'C4',12.:'C3',9.:'C2',20.:'C1',7.:'A2',5.:'A3',3.:'A1'}
adata.uns['cluster_names_dict'] = {str(k):v for k,v in cluster_names_dict.items()}
# Create a dictionary to map the cluster to a number
df['group_cluster_fancy'] = df.index.map(cluster_names_dict)
adata.obs['group_cluster_fancy'] = adata.obs['group_cluster'].map(cluster_names_dict)
# Also make a column that is the groups combined by A being Stem, B being Neural, C being Neutral, U being Undetermined, and NaN as Too Low
cluster_dict = {'A':'Stem', 'B':'Neural', 'C':'Neutral', 'U':'Undetermined', np.nan:'Too Low', 'a':'Stem', 'b':'Neural'}
for v in adata.obs['group_cluster_fancy'].unique():
    # if v is dtype float then it is NaN
    if type(v) == str:
        cluster_dict[v] = cluster_dict[v[0]]
# map the dict to the group_cluster_fancy column
adata.obs['group_cluster_neural'] = adata.obs['group_cluster_fancy'].map(cluster_dict)
# Create a dict for the group score
group_score_dict = dict(zip(df.index, df['neural_stem_score']))
adata.obs['group_cluster_score'] = adata.obs['group_cluster'].map(group_score_dict)
# Save the adata
adata.write_h5ad(f'arm.smallncRNAs.h5ad')
# Print the df
df

In [ ]:
def clusterPlots9(adata, umapgroup, output):
    # Define varibles
    umap1 = '_'.join([umapgroup,'umap1'])
    umap2 = '_'.join([umapgroup,'umap2'])
    cluster = '_'.join([umapgroup,'cluster'])
    point_size = 20
    # Subset the AnnData object to the umapgroup where not NaN (i.e. clustered data that wasn't filtered out)
    adata = adata[~adata.obs[cluster].isna(), :]
    # Create a list of clusters greater than or equal to 0 in size to filter out non-clustered reads
    hdbscan_annotated = adata.obs[cluster] >= 0
    neural_annotated = adata.obs['neural_broad'] == 'neural'
    stem_annotated = adata.obs['neural_broad'] == 'stem'
    neutral_annotated = adata.obs['neural_broad'] == 'neutral'
    # Create a 3 x 3 subplot with the umap projection and the cluster labels as the last subplot
    fig, axs = plt.subplots(3, 3, figsize=(20,20))
    # Plot first through ninth subplots
    plot_list = [('Amino Acid','amino',0,0), ('Tissue Type','tissuetype',0,1), ('AlkB Treatment','treatment',0,2), \
                 ('Timepoint','timepoint',1,0), ('Fragment Type','fragment',1,1), ('Total Number of Reads Unique','nreads_total_unique_norm',1,2), \
                 ('Neural Fine','neural_fine',2,0), ('Neural Broad','neural_broad',2,1), ('HDBScan',cluster,2,2)]
    for i in plot_list:
        if i[2] == 1 and i[3] == 2:
            pal = dict(zip(sorted(pd.unique(adata.obs[i[1]])), sns.color_palette("mako", len(pd.unique(adata.obs[i[1]])))))
        else:
            if i[1] == cluster:
                pal = dict(zip(sorted(pd.unique(adata.obs[i[1]][hdbscan_annotated])), sns.color_palette("hls", len(pd.unique(adata.obs[i[1]]))-1)))
            else:
                pal = dict(zip(sorted(pd.unique(adata.obs[i[1]])), sns.color_palette("hls", len(pd.unique(adata.obs[i[1]])))))
        # Sort the adata object by the categorical variable for legend purposes
        adata = adata[adata.obs[i[1]].sort_values().index, :]
        if i[1] == cluster:
            sns.scatterplot(x=adata.obs[umap1][~hdbscan_annotated], y=adata.obs[umap2][~hdbscan_annotated], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], color=np.array([(0.5,0.5,0.5)]), alpha=0.5, legend=False)
            sns.scatterplot(x=adata.obs[umap1][hdbscan_annotated], y=adata.obs[umap2][hdbscan_annotated], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]][hdbscan_annotated], palette=pal, legend=False)
            # Add labels to the plots
            for j in adata.obs[cluster][hdbscan_annotated].unique():
                x = adata.obs[umap1][adata.obs[cluster] == j].mean()
                y = adata.obs[umap2][adata.obs[cluster] == j].mean()
                # if umapgroup == 'group':
                    # name = adata.obs['group_cluster_fancy'][adata.obs[cluster] == j].unique()[0]
                # else:
                name = adata.obs[cluster][adata.obs[cluster] == j].unique()[0]
                axs[i[2],i[3]].text(x, y, name, fontsize=10, color='black', fontweight='bold')
        elif i[1] == 'neural_broad' or i[1] == 'neural_fine':
            sns.scatterplot(x=adata.obs[umap1][neutral_annotated], y=adata.obs[umap2][neutral_annotated], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]], palette=neural_color_dict, legend=False)
            sns.scatterplot(x=adata.obs[umap1][stem_annotated], y=adata.obs[umap2][stem_annotated], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]], palette=neural_color_dict, legend=False)
            sns.scatterplot(x=adata.obs[umap1][neural_annotated], y=adata.obs[umap2][neural_annotated], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]], palette=neural_color_dict, legend=False)
        else:
            sns.scatterplot(x=adata.obs[umap1], y=adata.obs[umap2], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]], palette=pal, legend=False)
        axs[i[2],i[3]].set_title(i[0])
        axs[i[2],i[3]].set_xlabel('UMAP 1')
        axs[i[2],i[3]].set_ylabel('UMAP 2')
        axs[i[2],i[3]].set_xticks([])
        axs[i[2],i[3]].set_yticks([])
        # Create legend from pal adding outside of plot and also reduce the size of the legend
        # if i[2] != 1 and i[3] != 2:
        #     axs[i[2],i[3]].legend(pd.unique(adata.obs[i[1]]), loc='upper right', bbox_to_anchor=(1.25, 1), frameon=False)
    # Add title
    fig.suptitle(f'UMAP Projection of tRNAs sorted by tRAX {umapgroup}', y=1.01)
    # Save figure
    plt.tight_layout()
    plt.savefig(output + f'/overview_9x_{umapgroup}.pdf')
    # plt.close()

def clusterPlots16(adata, umapgroup, output):
    # Define varibles
    umap1 = '_'.join([umapgroup,'umap1'])
    umap2 = '_'.join([umapgroup,'umap2'])
    cluster = '_'.join([umapgroup,'cluster'])
    point_size = 20
    # Subset the AnnData object to the umapgroup where not NaN (i.e. clustered data that wasn't filtered out)
    adata = adata[~adata.obs[cluster].isna(), :]
    # Create a list of clusters greater than or equal to 0 in size to filter out non-clustered reads
    hdbscan_annotated = adata.obs[cluster] >= 0
    # Create a 3 x 3 subplot with the umap projection and the cluster labels as the last subplot
    fig, axs = plt.subplots(4, 4, figsize=(24,24))
    # Plot first through ninth subplots
    plot_list = [('Amino Acid','amino',0,0), ('HDBScan',cluster,0,1), ('mods_6','mods_6',0,2), ('mods_7','mods_7',0,3), \
                 ('mods_9','mods_9',1,0), ('mods_10','mods_10',1,1), ('mods_14','mods_14',1,2), ('mods_20','mods_20',1,3), \
                 ('mods_20a','mods_20a',2,0), ('mods_26','mods_26',2,1), ('mods_27','mods_27',2,2), ('mods_32','mods_32',2,3), \
                 ('mods_34','mods_34',3,0), ('mods_37','mods_37',3,1), ('mods_e2','mods_e2',3,2), ('mods_58','mods_58',3,3)]

    for i in plot_list:
        # Determine wether to mask NaN values
        masking = False
        if adata.obs[i[1]].isna().any() or i[1] == cluster:
            masking = True
            if i[1] == cluster:
                # Create a list of clusters greater than or equal to 0 in size to filter out non-clustered reads from the HDBScan cluster
                mask = adata.obs[cluster] >= 0
            else:
                mask = ~adata.obs[i[1]].isnull()
        # Create a palette for the categorical variable
        if i[1] == cluster:
            pal = dict(zip(sorted(pd.unique(adata.obs[i[1]][hdbscan_annotated])), sns.color_palette("hls", len(pd.unique(adata.obs[i[1]]))-1)))
        else:
            if masking:
                pal = dict(zip(sorted(pd.unique(adata.obs[i[1]][mask])), sns.color_palette("hls", len(pd.unique(adata.obs[i[1]][mask])))))
            else:
                pal = dict(zip(sorted(pd.unique(adata.obs[i[1]])), sns.color_palette("hls", len(pd.unique(adata.obs[i[1]])))))
        # Sort the adata object by the categorical variable for legend purposes
        adata = adata[adata.obs[i[1]].sort_values().index, :]
        if masking:
            sns.scatterplot(x=adata.obs[umap1][~mask], y=adata.obs[umap2][~mask], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], color=np.array([(0.5,0.5,0.5)]), alpha=0.5, legend=False)
            sns.scatterplot(x=adata.obs[umap1][mask], y=adata.obs[umap2][mask], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]][mask], palette=pal, legend=False)
            if i[1] == cluster:
                # Add labels to the plots
                for j in adata.obs[cluster][hdbscan_annotated].unique():
                    x = adata.obs[umap1][adata.obs[cluster] == j].mean()
                    y = adata.obs[umap2][adata.obs[cluster] == j].mean()
                    if umapgroup == 'group':
                        name = adata.obs['group_cluster_fancy'][adata.obs[cluster] == j].unique()[0]
                    else:
                        name = adata.obs[cluster][adata.obs[cluster] == j].unique()[0]
                    axs[i[2],i[3]].text(x, y, name, fontsize=10, color='black', fontweight='bold')
        else:
            sns.scatterplot(x=adata.obs[umap1], y=adata.obs[umap2], s=point_size, linewidth=0.25, ax=axs[i[2],i[3]], hue=adata.obs[i[1]], palette=pal, legend=False)
        axs[i[2],i[3]].set_title(i[0])
        axs[i[2],i[3]].set_xlabel('UMAP 1')
        axs[i[2],i[3]].set_ylabel('UMAP 2')
        axs[i[2],i[3]].set_xticks([])
        axs[i[2],i[3]].set_yticks([])
        # Create legend from pal adding outside of plot and also reduce the size of the legend
        # if i[2] != 1 and i[3] != 2:
        #     axs[i[2],i[3]].legend(pd.unique(adata.obs[i[1]]), loc='upper right', bbox_to_anchor=(1.25, 1), frameon=False)
    # Add title
    fig.suptitle(f'UMAP Projection of tRNAs sorted by tRAX {umapgroup}', y=1.01)
    # Save figure
    plt.tight_layout()
    plt.savefig(output + f'/overview_mods_{umapgroup}.pdf')
    # plt.close()

adata = ad.read_h5ad('arm.smallncRNAs.h5ad')
os.makedirs('figures/arm.smallncRNAs/cluster/', exist_ok=True)
output = 'figures/arm.smallncRNAs/cluster'
# Plot the UMAP projection
# clusterPlots9(adata.copy(), 'sample', output)
clusterPlots9(adata.copy(), 'group', output)
# clusterPlots16(adata.copy(), 'sample', output)
clusterPlots16(adata.copy(), 'group', output)

In [ ]:
seqtype = 'arm.smallncRNAs'
# Generate the cluster overview plots
print('Generating cluster overview plots...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clusteroverview -o figures/{seqtype}/ 
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp group_cluster --clusterlabel group_cluster_fancy -o figures/{seqtype}/ --quiet

In [ ]:
seqtype = 'arm.smallncRNAs'
# Generate the cluster plots for discrete groups
plot_list = [('Amino Type','amino'), ('Tissue Type','tissuetype'), ('AlkB Treatment','treatment'), ('Timepoint','timepoint'), \
             ('Fragment Type','fragment'), ('Neural Fine','neural_fine'), ('Neural Broad','neural_broad')]
print('Generating cluster plots for discrete groups...')
for i in plot_list:
    print(i[0])
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp {i[1]} --clustermask -o figures/{seqtype}/ --quiet
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp {i[1]} -o figures/{seqtype}/ --quiet

In [ ]:
seqtype = 'arm.smallncRNAs'
# # Generate the cluster plots for continuous groups
# plot_list = [('hmm_score','hmm_score'), ('secstruct_score','secstruct_score'), ('cove_score','cove_score')]
plot_list = [('neural_stem_score','neural_stem_score')]
# print('Generating cluster plots for continuous groups...')
for i in plot_list:
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp {i[1]} --clusternumeric -o figures/{seqtype}/ #--quiet

In [ ]:
seqtype = 'arm.smallncRNAs'
# Generate the cluster plots for the neural_stem_score with HDBScan cluster labels
print('Generating cluster plots for the neural_stem_score...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp group_cluster_neural --clusterlabel group_cluster_fancy --clustermask -o figures/{seqtype}/
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g cluster --clustergrp group_cluster_fancy --clusterlabel group_cluster_fancy --clustermask -o figures/{seqtype}/

# 4. Custom visualizations

Plots to support conclusions in the paper, such as dot matrix plots, and kmer heatmaps.

## Modification Correlation Plot

The mod correlation plot is a dotplot of the correlation between the modification levels, the average neural-stem stem log2FC score, and the HDBSCAN group clusters.

In [ ]:
seqtype = 'arm.smallncRNAs'
adata = ad.read_h5ad(f'{seqtype}.h5ad')
cluster_names_dict = {float(k):v for k,v in adata.uns['cluster_names_dict'].items()}
# Create a df of the mods
mods_df = pd.DataFrame(adata.obs[['mods_6','mods_7','mods_9','mods_10','mods_14','mods_20','mods_20a','mods_26','mods_27','mods_32','mods_34','mods_37','mods_e2','mods_58']])
for mod in mods_df.columns:
    # Get unique mods for each mod
    mods = mods_df[mod].dropna().unique()
    for i in mods:
        mods_df[i+'_'+mod.split('_')[1]] = mods_df[mod].str.contains(i)
    # Drop the original mod column
    mods_df.drop(mod, axis=1, inplace=True)
# Add the group_cluster column from the adata.obs and the trna column from the adata.obs
mods_df['group_cluster'] = adata.obs['group_cluster']
# Convert all False to NaN
mods_df.replace(False, np.nan, inplace=True)
# Drop the NaN values from the mods
mods_df = mods_df[~mods_df['group_cluster'].isna()]
# Count the number of unique mods in each cluster
mods_df = mods_df.groupby(['group_cluster']).agg(['value_counts']).T
# Drop the the multiindex
mods_df = mods_df.droplevel(1)
mods_df = mods_df.T.droplevel(1)
# Convert group_cluster to int
mods_df.index.name = None
mods_df.index = mods_df.index.astype(int)
# Sort the columns by the index
mods_df = mods_df.sort_index(axis=0)
mods_df = mods_df.sort_index(axis=1)
# Replace NaN with 0
mods_df.fillna(0, inplace=True)

# Make a df of value counts for each group_cluster
group_cluster_df = pd.DataFrame(adata.obs['group_cluster'].value_counts())
# group_cluster_df = group_cluster_df.reset_index()
group_cluster_df.index = group_cluster_df.index.astype(int)
# Sort the df by the index
group_cluster_df = group_cluster_df.sort_values(by='group_cluster')
# Drop the index name
group_cluster_df.index.name = None
# combine the mods_df and group_cluster_df on the index
mods_df = mods_df.join(group_cluster_df)
# divide each column by the count of the group_cluster
for i in mods_df.columns[:-1]:
    mods_df[i] = mods_df[i]/mods_df['count']
# Drop the count column
mods_df.drop('count', axis=1, inplace=True)

# Melting the dataframe for corr plotting
corr_df = mods_df.copy()
corr_df['cluster'] = corr_df.index
corr_df = corr_df.melt(id_vars=['cluster'], var_name='mods', value_name='Modification Proportion')
# Sort the mods column by the number
corr_df['sortmods'] = corr_df['mods'].str.split('_').str[1]
# Convert the 20a to 20
corr_df['sortmods'] = corr_df['sortmods'].replace('20a','20.1')
# Convert e2 to 46
corr_df['sortmods'] = corr_df['sortmods'].replace('e2','46')
# Convert the sortmods to int
corr_df['sortmods'] = corr_df['sortmods'].astype(float)
corr_df = corr_df.sort_values(by=['cluster','sortmods','mods'])
neural_stem_score_dict = {}
for cluster in adata.obs['group_cluster'].unique():
    neural_stem_score_dict[cluster] = adata.obs[adata.obs['group_cluster'] == cluster]['neural_stem_score'].dropna().drop_duplicates().mean() #['group_cluster_neural_percent'].mean() 
corr_df['Neural Stem Score'] = corr_df['cluster'].map(neural_stem_score_dict)
# Since the mean of the neural_stem_score was taken, it needs to be scaled between 0 and 1 again
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
corr_df['Neural Stem Score'] = scaler.fit_transform(corr_df['Neural Stem Score'].values.reshape(-1,1))
# Since relplot always sorts the x-axis this needs to be done to sort the mods by the mean of the Neural Stem Score
# Sort df by mean score of Neural Stem Score
corr_df = corr_df.sort_values(by=['Neural Stem Score','sortmods','mods'])
corr_df = corr_df.reset_index(drop=True)
cluster_sort_list = corr_df['cluster'].unique()
# Create a dictionary to map the cluster to a number
cluster_dict = dict(zip(cluster_sort_list, range(0,len(cluster_sort_list))))
# Map the dictionary to the cluster column
corr_df['cluster'] = corr_df['cluster'].map(cluster_dict)
# Create a dictionary to map the number back to the cluster
cluster_dict_rev = dict(zip(range(0,len(cluster_sort_list)), cluster_sort_list))

# Map the mods to the fancy names
corr_df['mods'] = corr_df['mods'].map(adata.uns['mod_fancy_names'])
# Create a diverging color palette with 64 as the separation point value meaning 1/4 of the palette is used for each side
# cmap = sns.diverging_palette(255, 85, s=255, l=70, sep=32, as_cmap=True)
cmap = mpl.colors.LinearSegmentedColormap.from_list('blend', ['deepskyblue','silver','gold'])
# Plot the correlation
axs = sns.relplot(x='cluster', y='mods', hue='Neural Stem Score', size='Modification Proportion', data=corr_df, palette=cmap, edgecolor=".7", 
                  height=10, sizes=(0, 225), size_norm=(-.01, 1))
# Add a vertical lines to separate the clusters at the 1/4 and 3/4 range
axs.ax.axvline(2.5, color='black', linestyle='--')
axs.ax.axvline(17.5, color='black', linestyle='--')
# axs.map(plt.axvline, x=2.5, color='black', linestyle='--')
# axs.map(plt.axvline, x=17.5, color='black', linestyle='--')
# Add lines at 25% and 75% percentile
# axs.map(plt.axvline, x=corr_df[['cluster','Neural Stem Score']].describe(percentiles=[0.25]).loc[['25%']]['cluster'].values+0.5, color='black', linestyle='--')
# axs.map(plt.axvline, x=corr_df[['cluster','Neural Stem Score']].describe(percentiles=[0.75]).loc[['75%']]['cluster'].values-0.5, color='black', linestyle='--')
# Add labels to the points
axs.set(ylabel="Modification by Position", xlabel="Cluster Groups", aspect="equal")
axs.despine(left=True, bottom=True)
# Use cluster_names_dict to map the cluster number to the cluster name from earlier in the notebook
axs.set(xticks=np.arange(len(mods_df.index)), xticklabels=[cluster_names_dict[cluster_dict_rev[i]] for i in np.arange(0,len(mods_df.index))])
# Set title
plt.title('tRNA modification Prevalence by Cluster')
# Move legend outside of plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)
axs._legend.remove()
# Save figure
plt.tight_layout()
plt.savefig(f'figures/{seqtype}/cluster/mods_corr.pdf')

## Amino Correlation Plot

The amino correlation plot is a dotplot of the correlation between the isotype proportion, the average neural-stem stem log2FC score, and the HDBSCAN group clusters.

In [ ]:
seqtype = 'arm.smallncRNAs'
adata = ad.read_h5ad(f'{seqtype}.h5ad')
# Create a df of the aminos
aminos_df = pd.DataFrame(adata.obs['amino'])

for amino in aminos_df.columns:
    # Get unique amino acids for each column 
    aminos = aminos_df[amino].dropna().unique()
    for i in aminos:
        aminos_df[i] = aminos_df[amino].str.contains(i)
    # Drop the original mod column
    aminos_df.drop(amino, axis=1, inplace=True)
# Add the group_cluster column from the adata.obs and the trna column from the adata.obs
aminos_df['group_cluster'] = adata.obs['group_cluster']
# Drop Und and Sup from the aminos_df
# aminos_df.drop(['Und','Sup'], axis=1, inplace=True)
# Convert all False to NaN
aminos_df.replace(False, np.nan, inplace=True)
# Drop the NaN values from the aminos
aminos_df = aminos_df[~aminos_df['group_cluster'].isna()]
# Count the number of unique aminos in each cluster
aminos_df = aminos_df.groupby(['group_cluster']).agg(['value_counts']).T
# Drop the the multiindex
aminos_df = aminos_df.droplevel(1)
aminos_df = aminos_df.T.droplevel(1)
# Convert group_cluster to int
aminos_df.index.name = None
aminos_df.index = aminos_df.index.astype(int)
# Sort the columns by the index
aminos_df = aminos_df.sort_index(axis=0)
aminos_df = aminos_df.sort_index(axis=1)
# Replace NaN with 0
aminos_df.fillna(0, inplace=True)

# combine the aminos_df and group_cluster_df on the index
aminos_df = aminos_df.join(group_cluster_df)
# divide each column by the count of the group_cluster
for i in aminos_df.columns[:-1]:
    aminos_df[i] = aminos_df[i]/aminos_df['count']
# Drop the count column
aminos_df.drop('count', axis=1, inplace=True)

# Melting the dataframe for corr plotting
acorr_df = aminos_df.copy()
acorr_df['cluster'] = acorr_df.index
acorr_df = acorr_df.melt(id_vars=['cluster'], var_name='aminos', value_name='Isotype Proportion')
# Add the proportion of neural, neutral, and stem to the corr_df
acorr_df['Neural Stem Score'] = acorr_df['cluster'].map(neural_stem_score_dict)
# Since the mean of the neural_stem_score was taken, it needs to be scaled between -1 and 1 again
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
acorr_df['Neural Stem Score'] = scaler.fit_transform(acorr_df['Neural Stem Score'].values.reshape(-1,1))

acorr_df = acorr_df.sort_values(by=['cluster','aminos'])
# Map the dictionary to the cluster column
acorr_df['cluster'] = acorr_df['cluster'].map(cluster_dict)

# Create a diverging color palette with 64 as the separation point value meaning 1/4 of the palette is used for each side
# cmap = sns.diverging_palette(255, 85, s=255, l=70, sep=32, as_cmap=True)
cmap = mpl.colors.LinearSegmentedColormap.from_list('blend', ['deepskyblue','silver','gold'])
axs = sns.relplot(x='cluster', y='aminos', hue='Neural Stem Score', size='Isotype Proportion', data=acorr_df, palette=cmap, edgecolor=".7", 
                  height=10, sizes=(0, 225), hue_norm=(-1, 1), size_norm=(-.01, 1))
# Add a vertical lines to separate the clusters at the 1/4 and 3/4 range
# axs.map(plt.axvline, x=2.5, color='black', linestyle='--')
# axs.map(plt.axvline, x=17.5, color='black', linestyle='--')
axs.ax.axvline(2.5, color='black', linestyle='--')
axs.ax.axvline(17.5, color='black', linestyle='--')

axs.set(ylabel="Isotype", xlabel="Cluster Groups", aspect="equal")
axs.despine(left=True, bottom=True)
# Use cluster_names_dict to map the cluster number to the cluster name from earlier in the notebook
axs.set(xticks=np.arange(len(mods_df.index)), xticklabels=[cluster_names_dict[cluster_dict_rev[i]]  for i in np.arange(0,len(mods_df.index))])
# Set title
plt.title('tRNA Isotype Prevalence by Cluster')
# Move legend outside of plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)
axs._legend.remove()
# Save figure
plt.tight_layout()
plt.savefig(f'figures/{seqtype}/cluster/amino_corr.pdf')

## PCA Plot

This will generate a PCA plot that includes small RNA reads in addition to tRNA reads since the default PCA plot does not include small RNA reads.

In [ ]:
seqtype = 'arm.smallncRNAs'
adata = ad.read_h5ad(f'{seqtype}.h5ad')
os.makedirs(f'figures/{seqtype}/pca_alt/', exist_ok=True)

hue_dict = {'AlkBm_arm_d0_1': 'Day 0', 'AlkBm_arm_d0_2': 'Day 0', 'AlkBm_arm_d0_3': 'Day 0', 'AlkBm_arm_d14_1': 'Day 14', 'AlkBm_arm_d14_2': 'Day 14', 'AlkBm_arm_d35_1': 'Day 35', 'AlkBm_arm_d35_2': 'Day 35', 'AlkBm_arm_d35_3': 'Day 35', \
            'AlkBm_arm_d35_4': 'Day 35', 'AlkBm_arm_d35_5': 'Day 35', 'AlkBm_arm_d35_6': 'Day 35', 'AlkBm_arm_d70_1': 'Day 70', 'AlkBm_arm_d70_2': 'Day 70', 'AlkBm_arm_d70_3': 'Day 70', 'AlkBp_arm_d0_1': 'Day 0', 'AlkBp_arm_d0_2': 'Day 0', \
            'AlkBp_arm_d0_3': 'Day 0', 'AlkBp_arm_d0_4': 'Day 0', 'AlkBp_arm_d0_5': 'Day 0', 'AlkBp_arm_d0_6': 'Day 0', 'AlkBp_arm_d0_7': 'Day 0', 'AlkBp_arm_d0_8': 'Day 0', 'AlkBp_arm_d14_1': 'Day 14', 'AlkBp_arm_d14_2': 'Day 14', \
            'AlkBp_arm_d35_1': 'Day 35', 'AlkBp_arm_d35_2': 'Day 35', 'AlkBp_arm_d35_3': 'Day 35', 'AlkBp_arm_d35_4': 'Day 35', 'AlkBp_arm_d35_5': 'Day 35', 'AlkBp_arm_d70_1': 'Day 70', 'AlkBp_arm_d70_2': 'Day 70'}

marker_dict = {'AlkBm_arm_d0_1': 'AlkB-', 'AlkBm_arm_d0_2': 'AlkB-', 'AlkBm_arm_d0_3': 'AlkB-', 'AlkBm_arm_d14_1': 'AlkB-', 'AlkBm_arm_d14_2': 'AlkB-', 'AlkBm_arm_d35_1': 'AlkB-', 'AlkBm_arm_d35_2': 'AlkB-', 'AlkBm_arm_d35_3': 'AlkB-',\
               'AlkBm_arm_d35_4': 'AlkB-', 'AlkBm_arm_d35_5': 'AlkB-', 'AlkBm_arm_d35_6': 'AlkB-', 'AlkBm_arm_d70_1': 'AlkB-', 'AlkBm_arm_d70_2': 'AlkB-', 'AlkBm_arm_d70_3': 'AlkB-', 'AlkBp_arm_d0_1': 'AlkB+', 'AlkBp_arm_d0_2': 'AlkB+', \
               'AlkBp_arm_d0_3': 'AlkB+', 'AlkBp_arm_d0_4': 'AlkB+', 'AlkBp_arm_d0_5': 'AlkB+', 'AlkBp_arm_d0_6': 'AlkB+', 'AlkBp_arm_d0_7': 'AlkB+', 'AlkBp_arm_d0_8': 'AlkB+', 'AlkBp_arm_d14_1': 'AlkB+', 'AlkBp_arm_d14_2': 'AlkB+', \
               'AlkBp_arm_d35_1': 'AlkB+', 'AlkBp_arm_d35_2': 'AlkB+', 'AlkBp_arm_d35_3': 'AlkB+', 'AlkBp_arm_d35_4': 'AlkB+', 'AlkBp_arm_d35_5': 'AlkB+', 'AlkBp_arm_d70_1': 'AlkB+', 'AlkBp_arm_d70_2': 'AlkB+'}

df = pd.DataFrame()
for i in ['nreads_fiveprime_norm','nreads_threeprime_norm','nreads_other_norm','nreads_wholecounts_norm']:
    tdf = pd.DataFrame(adata.obs[['trna','sample',i]])
    # add i to all in tRNA column
    tdf['trna'] = tdf['trna'].astype(str) + '_' + i
    # pivot the table
    tdf = tdf.pivot(index='trna', columns='sample', values=i)
    # Remove index/column name
    tdf.index.name = None
    tdf.columns.name = None
    # Sort the columns
    tdf = tdf.reindex(sorted(tdf.columns), axis=1)
    # Combine the two df on matching columns
    df = pd.concat([df, tdf], axis=0)

# Sort the rows
df = df.sort_index()
tdf = adata.uns['nontRNA_counts'].reindex(sorted(adata.uns['nontRNA_counts'].columns), axis=1)
df = pd.concat([df, tdf], axis=0)
# Drop all rows with a sum less than 20
df = df[df.sum(axis=1) >= 20]

print(df)

# Standardize the data
df = pd.DataFrame(preprocessing.Normalizer().fit_transform(df), columns=df.columns, index=df.index)
# # Create a PCA object
pca = PCA(n_components=min(len(df.columns), 5))
pca.fit_transform(df)
evr = pca.explained_variance_ratio_

# Transform the data and create a new dataframe
pca_index = ['PC{}'.format(x) for x in range(1, len(evr)+1)]
df_pca = pd.DataFrame(pca.components_, columns=df.columns, index=pca_index).T
df_pca['hue'] = df_pca.index.map(hue_dict)
df_pca['marker'] = df_pca.index.map(marker_dict)


print('Principal components: {}'.format([f'PC{x}' for x in range(1, len(evr)+1)]))
print('Explained variance: {}'.format([f'{i:.4f}' for i in pca.explained_variance_]))
print('Explained variance ratio: {}'.format([f'{i*100:.2f}%' for i in evr]))

plt.figure(figsize=(6, 6))
ax = sns.barplot(x=['PC{}'.format(x) for x in range(1, len(evr)+1)], y=evr, palette=sns.hls_palette(len(evr)), hue=['PC{}'.format(x) for x in range(1, len(evr)+1)])
# Set the x and y labels and title
ax.set_xlabel('Principal Component')
ax.set_ylabel('Explained Variance Ratio')
ax.set_title('Explained Variance Ratio of Principal Components')
# Set the box aspect ratio to 1 so the plot is square
plt.gca().set_box_aspect(1)

plt.figure(figsize=(8, 8))
palette = {"Day 0": "#007FFF", "Day 14": "#00FF7F", "Day 35": "#FF007F", "Day 70": "#FFD700"}
ax = sns.scatterplot(data=df_pca, x='PC1', y='PC2', s=100, palette=palette, hue='hue', style='marker')
ax.set_xlabel('PC1 ({:.2f}%)'.format(evr[0]*100))
ax.set_ylabel('PC2 ({:.2f}%)'.format(evr[1]*100))
# Capatilize the legend and move the legend outside the plot and remove the border around it
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=[x.capitalize() for x in labels])
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), borderaxespad=0, frameon=False)
# ax.legend_.set_title(pcacolors.capitalize())
# Give the plot a title
# ax.set_title(f'PCA of {pcamarkers} colored by {pcacolors}')
# Remove the ticks and tick labels
ax.tick_params(axis='both', which='both', bottom=False, top=False,
                left=False, right=False, labelbottom=False, labelleft=False)
# Set the box aspect ratio to 1 so the plot is square
plt.gca().set_box_aspect(1)
plt.savefig(f'figures/{seqtype}/pca_alt/pca.pdf')

## K-mer Frequency Heatmap

The k-mer frequency heatmap is a heatmap of the k-mer frequency in the tRNA sequences, selected by top hits from k-mer enrichment analysis in neural and stem labeled tRNAs.

In [ ]:
adata = ad.read_h5ad('arm.smallncRNAs.h5ad')
rsfull_list = list(set(adata.obs['refseq'].tolist()))
kmers_occurance_dict = {}
kmer_freq_dict = {}

def kmerGen(seq, k):
    kmers = pd.Series([seq[x:x+k] for x in range(len(seq) - k + 1)])
    return kmers

for i in [3,4,5,6]:
    kmer_dict = {}
    for j in rsfull_list:
        tkmer = kmerGen(j, i)
        tkmer = tkmer[~tkmer.str.contains('-')] # drop kmers with gaps
        tkmer = tkmer.value_counts().to_dict()
        tdict = {k:kmer_dict.get(k,0)+v for k,v in tkmer.items()}
        kmer_dict.update(tdict)
    kmers_occurance_dict[i] = kmer_dict

for i in [3,4,5,6]:
    kmer_df = pd.DataFrame()
    kmer_dict = {}
    for j in rsfull_list:
        kmer_df = pd.concat([kmer_df, pd.DataFrame(kmerGen(j, i))], axis=1)
    for j,row in kmer_df.iterrows():
        tdict = kmer_df.loc[j].value_counts().to_dict()
        tdict = {k:tdict.get(k,0)/len(kmer_df.columns) for k in tdict}
        kmer_dict[j] = tdict
    kmer_freq_dict[i] = kmer_dict

topkmersize = 10

for i in [3,4,5,6]:
    t_df = pd.DataFrame(kmers_occurance_dict[i], index=['count']).T
    t_df = t_df.sort_values(by='count', ascending=False)
    print(i)
    print(t_df[0:topkmersize].index.tolist())
    print(t_df[0:topkmersize]['count'].tolist())
    print()

# Lists taken from below
Neural_l = ['TTCG', 'CCCC', 'GCAT', 'CCCGG', 'GGCC', 'CCGG', 'AGGC', 'TGTA', 'CCCG', 'TTCGA', 'CCC', 'GGC', 'ATG', 'GCA', 'CAC', 'GCC', 'CAT', 'GTA', 'CGA']
Neutral_l = ['TGGC', 'CGTG', 'GGCG', 'CGTGG', 'TTCG', 'CGAA', 'GTTCG', 'CGAG', 'GGCC', 'AGGC', 'GGC', 'CGA', 'CGT', 'TCG', 'TGGC', 'GCG', 'CGC', 'CGTG']
Stem_l = ['CCCGG', 'CCCG', 'CCGG', 'TGGT', 'ATTC', 'TCCC', 'GATTC', 'GATT', 'CGGG', 'TCCCG', 'CGG', 'TTC', 'CCG', 'GGT', 'CCC', 'ATT']

# given the three lists find the difference of the three lists
def difference(lst1, lst2, lst3): 
    return list(set(lst1) - set(lst2) - set(lst3))

print(difference(Neural_l, Neutral_l, Stem_l))

In [ ]:
seqtype = 'arm.smallncRNAs'
adata = ad.read_h5ad(f'{seqtype}.h5ad')
os.makedirs(f'figures/{seqtype}/kmer_freq/', exist_ok=True)

class kmerFreqCalc():
    def __init__(self, adata, kmer, plots, adata_grp, ccatail=True, pltshow=False, labels=None):
        self.adata = adata
        self.kmer = sorted(kmer)
        self.plots = plots
        self.adata_grp = adata_grp
        self.ccatail = ccatail
        self.pltshow = pltshow
        self.labels = labels
        self.vrange = 0.4
        self.cmap = sns.diverging_palette(30, 170, s=255, l=70, sep=64, as_cmap=True)
        self.total_list = self.kmerFreq(list(set(self.adata.obs['refseq'].tolist())))

    def main(self):
        # make fig and zxes with subplots equal to the number of plots
        fig = plt.figure(figsize=(20, 3.5*len(self.plots)))
        # Create 4 subplots with the first 3 stacked on top of each other and the last one spanning all 3 on the right
        hr = [1]*len(self.plots)+[0.075]
        gs = gridspec.GridSpec(len(self.plots)+1, 2, width_ratios=[1, 0.02], height_ratios=hr)
        total_df = pd.DataFrame()
        # Plot the heatmaps
        for order, grp in enumerate(self.plots):
            df = self.diffCalc(grp)
            self.plotHeatmap(df, grp, fig, plt.subplot(gs[order,0]))
            total_df = pd.concat([total_df, df], axis=1)
        
        if not self.labels:
            # Replace all NaN with 0
            total_df.fillna(0, inplace=True)
            # Make a label list for the heatmaps last column
            total_df = pd.DataFrame([total_df.max(axis=1).tolist(),total_df.min(axis=1).tolist()], index=['max','min']).T
            total_df['diff'] = total_df['max'] - total_df['min']
            total_df['diff'] = total_df['diff'].abs()
            # get index of the max column greater than or equal to 0.2
            self.labels = total_df[total_df['diff'] >= (self.vrange/2)].index.tolist()
            self.labels = [i+1 for i in self.labels] # Correct to be 1 based

        ax = plt.subplot(gs[len(self.plots)-1,0])
        self.labels = np.linspace(1, 73, 73, dtype=int)
        ax.set_xticks([i-0.5 for i in self.labels])
        ax.set_xticklabels([str(i) for i in self.labels])

        self.plotSchematic(plt.subplot(gs[-1, 0]))
        ax_cb = plt.subplot(gs[0, 1])
        # Create a legend in the right subplot for the heatmap
        ax_cb.imshow(np.linspace(0, 100, 101).reshape(-1, 1)[::-1], cmap=self.cmap, aspect='auto', interpolation='nearest')
        ax_cb.set_xticks([])
        ax_cb.set_yticks([0, 25, 50, 75, 100])
        ax_cb.set_yticklabels([f'>={self.vrange*100}%', f'{self.vrange*50}%', '0%', f'-{self.vrange*50}%', f'<=-{self.vrange*100}%'])
        ax_cb.set_ylabel("Frequency from Background", rotation=90, verticalalignment='center')  
        ax_cb.yaxis.set_label_position("right")
        # Add title
        fig.suptitle(f'Kmer Frequency Difference from Total for {self.kmer}')
        # Save figure
        plt.tight_layout()
        plt.savefig(f'figures/{seqtype}/kmer_freq/{"_".join([str(i) for i in self.plots])}_{"_".join(self.kmer)}.pdf')
        if not self.pltshow:
            plt.close()

    def kmerGen(self, seq, ksize):
        return pd.Series([seq[x:x+ksize] for x in range(len(seq) - ksize + 1)])
    
    def kmerFreq(self, klist):
        kmer_dict = {}
        if not self.ccatail:
            klist = [i[0:73] for i in klist]
        for i in list(set([len(i) for i in self.kmer])):
            kmer_df = pd.DataFrame()
            for j in klist:
                kmer_df = pd.concat([kmer_df, pd.DataFrame(self.kmerGen(j, i))], axis=1)
            for j,row in kmer_df.iterrows():
                tdict = kmer_df.loc[j].value_counts().to_dict()
                tdict = {k:tdict.get(k,0)/len(kmer_df.columns) for k in tdict}
                kmer_dict[j] = kmer_dict.get(j,{})
                kmer_dict[j].update(tdict)
        return kmer_dict

    def diffCalc(self, grp):
        sub_list = self.adata.obs[self.adata.obs[self.adata_grp] == grp]['refseq'].tolist()
        # if not self.ccatail:
        #     sub_list = [i[0:73] for i in sub_list]
        sl = self.kmerFreq(sub_list)
        # Calc the difference between the total and the subset based on the kmer where if the number isnt in the subset it is 0
        diff_dict = {}
        for i in self.total_list:
            # diff_dict[i] = {k:np.abs(sl[i].get(k,tl[i].get(k,0))-tl[i].get(k,0)) for k in tl[i]}
            diff_dict[i] = {k:sl[i].get(k,self.total_list[i].get(k,0))-self.total_list[i].get(k,0) for k in self.total_list[i]}
        # create a df of the enrichment of the specific kmer in the subset
        if not self.ccatail:
            df = pd.DataFrame(np.zeros((73,1))) # Add this column to ensure that the df is created with the correct trna length
        else:
            df = pd.DataFrame(np.zeros((76,1))) 
        df = pd.concat([df, pd.DataFrame(diff_dict).T], axis=1)
        # drop the first column of zeros
        df.drop(0, axis=1, inplace=True)
        # Sort the columns alphabetically
        df = df.sort_index(axis=1)
        # Replace all NaN with 0
        df.fillna(0, inplace=True)
        # Print the top canidates for each group
        # print(grp)
        # print(df.sum(axis=0).sort_values(ascending=False)[0:10].index.tolist())
        # if self.kmer not in df.columns then add it with all NaN
        for i in self.kmer:
            if i not in df.columns:
                df[i] = np.nan
        # drop all columns that arent the kmer
        df = df[self.kmer]
        # for each row conver the n last columns to NaN where n is the length of the index name
        mask = np.zeros_like(df.T, dtype=bool)
        for i,row in enumerate(df.T.index):
            mask[i,-(len(row)-1):] = True
        df = df.T.mask(mask)
        # Sort the columns by length then alphabetically
        df['sort_1'] = df.index.str.len()
        df['sort_2'] = df.index
        df = df.sort_values(by=['sort_1','sort_2'])
        df.drop(['sort_1','sort_2'], axis=1, inplace=True)
        df = df.T    
        
        return df
    
    def plotHeatmap(self, df, grp, fig, ax):
        # Create a 0.33 array as background
        if not self.ccatail:
            zero_df = pd.DataFrame(np.full((73, len(df.columns)), 0.33), columns=df.columns)
        else:
            zero_df = pd.DataFrame(np.full((76, len(df.columns)), 0.33), columns=df.columns)
        ax = sns.heatmap(zero_df.T, cmap="Greys", linewidths=.5, ax=ax, cbar=False, vmin=0, vmax=1)
        # generate a heatmap of the df and return the ax
        ax = sns.heatmap(df.T, cmap=self.cmap, linewidths=.5, ax=ax, cbar=False, vmin=-1*self.vrange, vmax=self.vrange)
        # Plot dashed lines
        for j in [7, 9, 25, 26, 43, 48, 65]:
            ax.plot([j, j], [ax.get_ylim()[0], ax.get_ylim()[1]], color='k', linewidth=1, linestyle='--', zorder=1)
        # Remove the x and y tick labels
        ax.set_xticks([])
        ax.set_xticklabels([])      
        # Add title
        ax.set_title(f'Cluster {grp}')
        ax.set_xlim([0, 76])
        if not self.ccatail:
            ax.set_xlim([0, 73])
        
    def plotSchematic(self, ax):
        # Create the tRNA scheme in the bottom subplot
        ax.set_ylim([-1, 1])
        ax.set_xlim([0, 76])
        if not self.ccatail:
            ax.set_xlim([0, 73])
        ax.axhspan(-0.125, 0.125, color='black', zorder=-3)
        # Plot the tRNA scheme
        ax.axvspan(9, 25, color='#ff9999', zorder=-2) # D-Arm/Loop Outer
        ax.axvspan(13, 21, color='white', alpha=0.8, zorder=-1) # D-Arm/Loop Inner
        ax.axvspan(26, 43, color='#99ff99', zorder=-2) # A-Arm/Loop Outter
        ax.axvspan(31, 38, color='white', alpha=0.8, zorder=-1) # A-Arm/Loop Inner
        ax.axvspan(48, 65, color='#99ffff', zorder=-2) # T-Arm/Loop Outter
        ax.axvspan(53, 60, color='white', alpha=0.8, zorder=-1) # T-Arm/Loop Inner
        ax.axvspan(0, 7, color='#98c0c0', zorder=-2) # Stem 1
        if not self.ccatail:
            ax.axvspan(65, 73, color='#98c0c0', zorder=-2)
        else:
            ax.axvspan(65, 76, color='#98c0c0', zorder=-2) # Stem 2
        # # Plot the tRNA scheme text
        ax.text(17, 0, 'D-Arm/Loop', fontsize=10, verticalalignment='center', horizontalalignment='center', color='k')
        ax.text(35, 0, 'A-Arm/Loop', fontsize=10, verticalalignment='center', horizontalalignment='center', color='k')
        ax.text(56, 0, 'T-Arm/Loop', fontsize=10, verticalalignment='center', horizontalalignment='center', color='k')
        # Plot dashed lines
        for j in [7, 9, 25, 26, 43, 48, 65]:
            ax.plot([j, j], [ax.get_ylim()[0], ax.get_ylim()[1]], color='k', linewidth=1, linestyle='--', zorder=1)
        # Remove the axis spines for ax3
        ax.axis('off')

label_list = None
kmer_list =  ['CCCC', 'GGGG', 'TGTA', 'CAC', 'ATG', 'GTA', 'GCC', 'GCAT']  

adata_grp = 'group_cluster_neural'
plot = sorted(adata.obs[adata_grp].unique().tolist())
# remove the 'Too Low' and 'Undetermined' from the plot list
plot.remove('Too Low')
# plot.remove('Undetermined')
kmerFreqCalc(adata, kmer_list, plot, adata_grp, pltshow=True, ccatail=False, labels=label_list).main()

## Small RNA-seq Barplots

These plots require `statannotations` in order to run however at this time the dependencies for that require `Seaborn<=0.12` which causes issues with other plots in this pipeline. These plots should be generated manually but the code is provided for reference.

```Python
from statannotations.Annotator import Annotator

def rna_barplot(df, readtype, seqtype):
    # Subset the df
    df = df[df['readtype'] == readtype]
    # Define Pallete
    pal = {'d0':'#007FFF','d14':'#00FF7F','d35':'#FF007F','d70':'#FFD700'}
    test_type = 't-test_welch'
    # Creat fig
    fig, axs = plt.subplots(figsize=(6,5))
    sns.barplot(data=df, x='treatment', y='count', hue='timepoint', palette=pal, ax=axs)
    # Add statistical annotation
    annotator = Annotator(axs, pairs=[('AlkBp','AlkBm')], data=df, x='treatment', y='count', order=['AlkBp','AlkBm'])
    annotator.configure(test=test_type, text_format='star', loc='inside', verbose=0)
    annotator.apply_and_annotate()
    # Y axis label
    axs.set_ylabel('Normalized Reads')
    # X axis label
    axs.set_xlabel('')
    # Add title
    plt.title(f'{readtype} Reads')
    # Move legend outside of plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Timepoint', frameon=False)
    # Save figure
    plt.tight_layout()
    plt.savefig(f'figures/{seqtype}/bar/treatment_{readtype}_compare_{test_type}.pdf')
    plt.close()

    # Creat fig
    fig, axs = plt.subplots(figsize=(6,5))
    sns.barplot(data=df, x='group', y='count', hue='timepoint', palette=pal, ax=axs)
    # Add statistical annotation
    ppairs = [('d0_AlkBm','d14_AlkBm'),('d0_AlkBm','d35_AlkBm'),('d0_AlkBm','d70_AlkBm'),('d0_AlkBp','d14_AlkBp'),('d0_AlkBp','d35_AlkBp'),('d0_AlkBp','d70_AlkBp')]
    oorder = ['d0_AlkBm','d14_AlkBm','d35_AlkBm','d70_AlkBm','d0_AlkBp','d14_AlkBp','d35_AlkBp','d70_AlkBp']
    annotator = Annotator(axs, pairs=ppairs, data=df, x='group', y='count', order=oorder)
    annotator.configure(test=test_type, text_format='star', loc='inside', verbose=0)
    annotator.apply_and_annotate()
    # Y axis label
    axs.set_ylabel('Normalized Reads')
    # X axis label
    axs.set_xlabel('')
    # Remove xticks and only have the two for the groups
    if seqtype == 'arm.smallncRNAs':
        axs.set_xticks([1.5,5.5])
    else:
        axs.set_xticks([0.5,2.5])
    axs.set_xticklabels(['AlkBm','AlkBp'])
    # Add title
    plt.title(f'{readtype} Reads')
    # Move legend outside of plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Timepoint', frameon=False)
    # Save figure
    plt.tight_layout()
    plt.savefig(f'figures/{seqtype}/bar/timepoint_{readtype}_compare_{test_type}.pdf')
    plt.close()

for seqtype in seqtypes:
    os.makedirs(f'figures/{seqtype}/bar', exist_ok=True)
    adata = ad.read_h5ad(f'{seqtype}.h5ad')
    df = adata.uns['type_real_counts'].T
    # Get readtypes for plots
    readtypes = sorted(df.columns)
    # Split the names into group and timepoint
    df['treatment'] = [i.split('_')[0] for i in df.index]
    df['timepoint'] = [i.split('_')[2] for i in df.index]
    df['sample'] = df.index
    df['group'] = df['timepoint'] + '_' + df['treatment']
    # Melt the df for bar plots
    df = pd.melt(df, id_vars=['treatment','timepoint','sample','group'], value_vars=readtypes, var_name='readtype', value_name='count')

    for rt in readtypes:
        rna_barplot(df, rt, seqtype)
```

# 5. Bulk visualizations

The following code will run the graphing part of the pipeline with the following configuration to subset the data into pAlkB samples:

Generate all the main plots

In [ ]:
for seqtype in seqtypes:
    for config in ['','--config config/AlkBp.json']:
        print(f'Generating figures for {seqtype} with {config}...')
        !../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad {config} --colormap config/colormap.json -g correlation count heatmap pca radar volcano -o figures/{seqtype}/ \
          --heatgrp timepoint --heatsubplots --heatcutoff 55 --heatbound 15 --bargrp timepoint --covgrp timepoint --radargrp timepoint --radarscaled --radarmethod all --corrgroup group --quiet

Generate specific bar plots

In [ ]:
seqtype = 'arm.smallncRNAs'
# Create standard bar plots
print(f'Generating standard bar plots for {seqtype}...')
for i in [('amino','timepoint'),('iso','timepoint')]:
    !../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g bar --bargrp {i[0]} --barcol {i[1]} -o figures/{seqtype}/ --quiet
# Create sub bar plots
print(f'Generating sub bar plots for {seqtype}...')
for i in [('iso','amino','timepoint'),('trna','iso','timepoint')]:
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g bar --bargrp {i[0]} --barsubgrp {i[1]} --barcol {i[2]} -o figures/{seqtype}/ --quiet


In [ ]:
# Create sorted by neural category bar plots for ARMseq
print('Generating sorted by neural and epi category bar plots for ARMseq...')
for i in [('neural_broad','group_cluster'),('treatment','group_cluster'),('timepoint','group_cluster')]:
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g bar --bargrp {i[0]} --barcol {i[1]} --barsort 'group_cluster_score' --barlabel 'group_cluster_fancy' -o figures/{seqtype}/ #--quiet
print('Done!')

Generate specific combined coverage plots

In [ ]:
seqtype = 'arm.smallncRNAs'
for cg,co in [('timepoint','iso'), ('timepoint','trna')]:
    print(f'Generating combined coverage figures for ARMseq for {cg} by {co}...')
    # ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --config config/AlkBp.json --colormap config/colormap.json -g coverage --covgrp {cg} --covobs {co} --covmethod mean -o figures/{seqtype}/ --covtype mismatchedbases --quiet
    # ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --config config/AlkBp.json --colormap config/colormap.json -g coverage --covgrp {cg} --covobs {co} --covmethod mean -o figures/{seqtype}/ --covtype readstarts --quiet
    ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --config config/AlkBp.json --colormap config/colormap.json -g coverage --covgrp {cg} --covobs {co} --covmethod mean -o figures/{seqtype}/ --covtype uniquecoverage --quiet
    # ! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --config config/AlkBp.json --colormap config/colormap.json -g coverage --covgrp {cg} --covobs {co} --covmethod sum -o figures/{seqtype}/ --covtype uniquecoverage --quiet

print('Generating coverage figures for ARMseq for group_cluster...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g coverage --covgrp group_cluster_fancy --covobs group_cluster_fancy --covmethod mean -o figures/{seqtype}/ --quiet
print('Done!')

Generate specific seqLogos and seqmaps for ARMseq data

In [ ]:
seqtype = 'arm.smallncRNAs'

adata = ad.read_h5ad(f'{seqtype}.h5ad')
# Generate logo plots for ARMseq based on group_cluster
print('Generating logo plots for ARMseq based on group_cluster...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logogrp group_cluster -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 

neural_clusters = [7,18,19,12,1]
stem_clusters = [6,2,10,8,9,3]

print('Generating logo plots for ARMseq based on tRNA-Arg-TCT-1 and tRNA-Arg-TCT-4...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-Arg-TCT-1 tRNA-Arg-TCT-2 tRNA-Arg-TCT-3 tRNA-Arg-TCT-4 tRNA-Arg-TCT-5 --logomanualname Arg-TCT -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-Arg-TCT-1 tRNA-Arg-TCT-4 --logomanualname Arg-TCT-1_4 -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-Arg-TCT-4 --logomanualname Arg-TCT-4 -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 

tlist = []
for i in neural_clusters:
    tlist.append(adata.obs[adata.obs['group_cluster'].isin(neural_clusters)]['trna'].unique().tolist())
tlist = sorted(set(tlist[0]))
tlist = ' '.join(tlist)
tname = 'neural_clusters'
    
print('Generating logo plots for ARMseq based on neural_clusters...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json --config config/AlkBp.json -g logo --logomanualgrp {tlist} --logomanualname {tname} -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 

tlist = []
for i in stem_clusters:
    tlist.append(adata.obs[adata.obs['group_cluster'].isin(stem_clusters)]['trna'].unique().tolist())
tlist = sorted(set(tlist[0]))
tlist = ' '.join(tlist)
tname = 'stem_clusters'

print('Generating logo plots for ARMseq based on stem_clusters...')
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp {tlist} --logomanualname {tname} -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 

# Based on the top 15 neural expression from differential expression (Fig 2)
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-SeC-TCA-1 tRNA-Arg-TCT-4 tRNA-Ala-AGC-8 tRNA-Gly-CCC-2 tRNA-Leu-AAG-5 \
    tRNA-Leu-TAG-3 tRNA-Gly-GCC-2 tRNA-Arg-CCT-4 tRNA-Leu-AAG-2 tRNA-Ser-TGA-1 tRNA-Arg-CCT-2 tRNA-Ala-AGC-4 tRNA-Ser-GCT-3 tRNA-Ala-TGC-3 tRNA-Ala-TGC-4 \
    --logomanualname neural_top_15 -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-SeC-TCA-1 tRNA-Arg-TCT-4 tRNA-Ala-AGC-8 tRNA-Gly-CCC-2 tRNA-Leu-AAG-5 \
    tRNA-Leu-TAG-3 tRNA-Gly-GCC-2 tRNA-Arg-CCT-4 tRNA-Leu-AAG-2 tRNA-Ser-TGA-1 tRNA-Arg-CCT-2 tRNA-Ala-AGC-4 tRNA-Ser-GCT-3 \
    --logomanualname neural_top_15_13_subset -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet 
! ../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad --colormap config/colormap.json -g logo --logomanualgrp tRNA-Ala-TGC-3 tRNA-Ala-TGC-4 \
    --logomanualname neural_top_15_2_subset -o figures/{seqtype}/ --logosize sprinzl --ccatail --logornamode --quiet

In [ ]:
for seqtype in seqtypes:
    for config in ['--config config/AlkBp.json']:
        print(f'Generating figures for {seqtype} with {config}...')
        !../tRNAgraph/trnagraph.py graph -i {seqtype}.h5ad {config} --colormap config/colormap.json -g radar -o figures/{seqtype}/ --radargrp timepoint --radarscaled --radarmethod mean

# 6. Supplemental data analysis

Cerebral organoid expression from RNAseq data is used and correlated. `1-s2.0-S0092867418303830-mmc2.csv` supplementary data from [https://doi.org/10.1016/j.cell.2018.03.051](https://doi.org/10.1016/j.cell.2018.03.051) was loaded and used to plot the expression of tRNA modification enzymes in the RNAseq data.

In [ ]:
df = pd.read_csv('supplemental/1-s2.0-S0092867418303830-mmc2.csv')

# Add mean reads across all timepoints
df['mean_reads'] = df[['baseMean_hESw0','baseMean_hESw1','baseMean_hESw2','baseMean_hESw3','baseMean_hESw4','baseMean_hESw5']].mean(axis=1)

# Add log2diff between the 0 and 5 timepoints
df['log2diff'] = np.log2(df['baseMean_hESw5'] + 1) - np.log2(df['baseMean_hESw0'] + 1)

x = np.array([i for i in range(6)])
y = df[['baseMean_hESw0','baseMean_hESw1','baseMean_hESw2','baseMean_hESw3','baseMean_hESw4','baseMean_hESw5']].values
slopes, intercepts = [],[]

for i in range(0, df.shape[0]):
    slope, intercept = np.polyfit(x, y[i], 1)
    slopes.append(slope)
    intercepts.append(intercept)

df['slope'] = slopes
df['intercept'] = intercepts
# Normalize the slope between -1 and 1
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
df['slope_scaled'] = scaler.fit_transform(df['slope'].values.reshape(-1,1))
# Sort the df by the slope_scaled
df = df.sort_values(by='slope_scaled', ascending=False)
# Reset the index
df = df.reset_index(drop=True)
df

tRNA modification and disease data was obtained from from de [Crécy-Lagard et al. 2019](https://pmc.ncbi.nlm.nih.gov/articles/PMC6412123/).

In [ ]:
df_disease = pd.read_csv('supplemental/mods_disease.tsv', sep='\t')
# Fill NaN with False
df_disease.fillna(False, inplace=True)
# Drop Non tRNA causes
df_disease = df_disease[df_disease['Non tRNA cause'] == False]
diseses_list = ['microcephaly','developmental delay','intellectual disabilities','developmental defects','cmt','als','autism spectrum disorder','molybdenum cofactor deficiency','friedreich ataxia','galloway-mowat syndrome','neurodegeneration','encephalopathy','leigh syndrome','epilepsy',]
disease_genes = []
for i in diseses_list:
    disease_genes += df_disease[df_disease[i] == True]['Gene'].tolist()
disease_genes = list(set(disease_genes))
df_disease

In [ ]:
df_sub = pd.DataFrame()

# print(disease_genes)
# Disease list
target_genes = ['HSD17B10','PUS10','OSGEP','LAGE3','TRMT1','PUS3','ADAT3','FTSJ1','ELP4','TPRKB','TRMT10A','TRIT1', 'THG1L']
# Position list
target_genes += ['ADAT1','ADAT3','DTWD1','DTWD2','NSUN2','PUS1','PUS3','TRMT1','TRMT61A','TRMT10A','METTL6']
print(target_genes)

for i in list(set(target_genes)):
    df_sub = pd.concat([df_sub, df[df['gene_symbol']==i]])

# target_genes = ['NSUN','PUS3','PUS1','TRMT1','ADAT3','TRMT6','ACP3U','DTWD1','DTWD2','TRMT5','TRMT61A','ADAT1']
# for i in list(set(disease_genes + target_genes)):
#     df_sub = pd.concat([df_sub, df[df['gene_symbol']==i]])

df_sub = df_sub[['gene_symbol','baseMean_hESw0','baseMean_hESw1','baseMean_hESw2','baseMean_hESw3','baseMean_hESw4','baseMean_hESw5']]
df_sub = df_sub.set_index('gene_symbol')


df_sub = pd.DataFrame(preprocessing.Normalizer(norm='l2').fit_transform(df_sub),columns=df_sub.columns,index=df_sub.index)

# Sort the df by baseMean_hESw0 then baseMean_hESw5
df_sub['diff'] = df_sub['baseMean_hESw5'] - df_sub['baseMean_hESw0']
df_sub = df_sub.sort_values(by='diff', ascending=False)
df_sub.drop('diff', axis=1, inplace=True)
# Sort by gene_sumbol
# df_sub = df_sub.sort_index()

fig, axs = plt.subplots(figsize=(12,5))
sns.heatmap(df_sub[['baseMean_hESw0','baseMean_hESw1','baseMean_hESw2','baseMean_hESw3','baseMean_hESw4','baseMean_hESw5']].T, annot=False, cmap='mako_r',  ax=axs)
# Remove the yticklabels
axs.set_yticklabels(['Week 0','Week 1','Week 2','Week 3','Week 4','Week 5'], rotation=0)
axs.set_ylabel('Timepoint')
# Add title
plt.title('tRNA Expression in hESCs')
# Save figure
plt.tight_layout()
plt.savefig(f'figures/hESC_tRNA_expression.pdf')

# 7. CSV exportation

In [ ]:
for seqtype in ['arm.smallncRNAs']:
    !../tRNAgraph/trnagraph.py tools csv -i {seqtype}.h5ad